In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append("..")

In [3]:
from optimus import Optimus

In [4]:
# Create optimus
op = Optimus()


                             ____        __  _                     
                            / __ \____  / /_(_)___ ___  __  _______
                           / / / / __ \/ __/ / __ `__ \/ / / / ___/
                          / /_/ / /_/ / /_/ / / / / / / /_/ (__  ) 
                          \____/ .___/\__/_/_/ /_/ /_/\__,_/____/  
                              /_/                                  
                              
Transform and Roll out...
Just checking that all necessary environments vars are present...
-----
PYSPARK_PYTHON=python
SPARK_HOME=C:\opt\spark\spark-2.3.1-bin-hadoop2.7
JAVA_HOME=C:\java8
Pyarrow Installed
-----
Starting or getting SparkSession and SparkContext...
Setting checkpoint folder local. If you are in a cluster initialize Optimus with master='your_ip' as param
Deleting previous folder if exists...
Creating the checkpoint directory...
Optimus successfully imported. Have fun :).


## Create dataframe
### Spark

This is ugly:

```
val someData = Seq(
  Row(8, "bat"),
  Row(64, "mouse"),
  Row(-27, "horse")
)

val someSchema = List(
  StructField("number", IntegerType, true),
  StructField("word", StringType, true)
)

val someDF = spark.createDataFrame(
  spark.sparkContext.parallelize(someData),
  StructType(someSchema)
)```

In [5]:
from pyspark.sql.types import StructType, StructField, StringType, BooleanType, IntegerType, ArrayType

df = op.create.df(
            [
                ("words", "str", True),
                ("num", "int", True),
                ("animals", "str", True),
                ("thing", StringType(), True),
                ("two strings", StringType(), True),
                ("filter", StringType(), True),
                ("num 2", "string", True),
                ("col_array",  ArrayType(StringType()), True),
                ("col_int",  ArrayType(IntegerType()), True)

            ]
,
[
                ("  I like     fish  ", 1, "dog", "housé", "cat-car", "a","1",["baby", "sorry"],[1,2,3]),
                ("    zombies", 2, "cat", "tv", "dog-tv", "b","2",["baby 1", "sorry 1"],[3,4]),
                ("simpsons   cat lady", 2, "frog", "table","eagle-tv-plus","1","3", ["baby 2", "sorry 2"], [5,6,7]),
                (None, 3, "eagle", "glass", "lion-pc", "c","4", ["baby 3", "sorry 3"] ,[7,8])
            ])

df.table()

words (string),num (int),animals (string),thing (string),two strings (string),filter (string),num 2 (string),col_array (array),col_int (array)
⸱⸱I⸱like⸱⸱⸱⸱⸱fish⸱⸱,1,dog,housé,cat-car,a,1,"['baby',⸱'sorry']","[1,⸱2,⸱3]"
⸱⸱⸱⸱zombies,2,cat,tv,dog-tv,b,2,"['baby⸱1',⸱'sorry⸱1']","[3,⸱4]"
simpsons⸱⸱⸱cat⸱lady,2,frog,table,eagle-tv-plus,1,3,"['baby⸱2',⸱'sorry⸱2']","[5,⸱6,⸱7]"
None,3,eagle,glass,lion-pc,c,4,"['baby⸱3',⸱'sorry⸱3']","[7,⸱8]"


table function took 2456.864 ms


## Create Columns
### Spark
* You can not create multiple columns at the same time
* You need to use the lit function. lit???

### Pandas
* Assing function seems to do the job https://stackoverflow.com/questions/12555323/adding-new-column-to-existing-dataframe-in-python-pandas


In [6]:
df = df.cols.append("new_col_1", 1)
df.table()

append function took 318.165 ms


words (string),num (int),animals (string),thing (string),two strings (string),filter (string),num 2 (string),col_array (array),col_int (array),new_col_1 (int)
⸱⸱I⸱like⸱⸱⸱⸱⸱fish⸱⸱,1,dog,housé,cat-car,a,1,"['baby',⸱'sorry']","[1,⸱2,⸱3]",1
⸱⸱⸱⸱zombies,2,cat,tv,dog-tv,b,2,"['baby⸱1',⸱'sorry⸱1']","[3,⸱4]",1
simpsons⸱⸱⸱cat⸱lady,2,frog,table,eagle-tv-plus,1,3,"['baby⸱2',⸱'sorry⸱2']","[5,⸱6,⸱7]",1
None,3,eagle,glass,lion-pc,c,4,"['baby⸱3',⸱'sorry⸱3']","[7,⸱8]",1


table function took 1603.297 ms


In [7]:
from pyspark.sql.functions import *

df.cols.append([
    ("new_col_2", 2.22),
    ("new_col_3", lit(3))
    ]).table()

append function took 7.977 ms
append function took 2.992 ms
append function took 10.969 ms


words (string),num (int),animals (string),thing (string),two strings (string),filter (string),num 2 (string),col_array (array),col_int (array),new_col_1 (int),new_col_2 (double),new_col_3 (int)
⸱⸱I⸱like⸱⸱⸱⸱⸱fish⸱⸱,1,dog,housé,cat-car,a,1,"['baby',⸱'sorry']","[1,⸱2,⸱3]",1,2.22,3
⸱⸱⸱⸱zombies,2,cat,tv,dog-tv,b,2,"['baby⸱1',⸱'sorry⸱1']","[3,⸱4]",1,2.22,3
simpsons⸱⸱⸱cat⸱lady,2,frog,table,eagle-tv-plus,1,3,"['baby⸱2',⸱'sorry⸱2']","[5,⸱6,⸱7]",1,2.22,3
None,3,eagle,glass,lion-pc,c,4,"['baby⸱3',⸱'sorry⸱3']","[7,⸱8]",1,2.22,3


table function took 1708.787 ms


In [8]:
from pyspark.sql.functions import *

df.cols.append([
    ("new_col_4", "test"),
    ("new_col_5", df['num']*2),
    ("new_col_6", [1,2,3])
    ]).table()

append function took 7.017 ms
append function took 4.018 ms
append function took 23.893 ms
append function took 36.902 ms


words (string),num (int),animals (string),thing (string),two strings (string),filter (string),num 2 (string),col_array (array),col_int (array),new_col_1 (int),new_col_4 (string),new_col_5 (int),new_col_6 (array)
⸱⸱I⸱like⸱⸱⸱⸱⸱fish⸱⸱,1,dog,housé,cat-car,a,1,"['baby',⸱'sorry']","[1,⸱2,⸱3]",1,test,2,"[1,⸱2,⸱3]"
⸱⸱⸱⸱zombies,2,cat,tv,dog-tv,b,2,"['baby⸱1',⸱'sorry⸱1']","[3,⸱4]",1,test,4,"[1,⸱2,⸱3]"
simpsons⸱⸱⸱cat⸱lady,2,frog,table,eagle-tv-plus,1,3,"['baby⸱2',⸱'sorry⸱2']","[5,⸱6,⸱7]",1,test,4,"[1,⸱2,⸱3]"
None,3,eagle,glass,lion-pc,c,4,"['baby⸱3',⸱'sorry⸱3']","[7,⸱8]",1,test,6,"[1,⸱2,⸱3]"


table function took 1623.539 ms


## Select columns
### Spark
* You can not select columns by string and index at the same time

### Pandas
* You can not select columns by string and index at the same time

In [66]:
df.table()
columns = ["words", 1, "animals", 3]
df.cols.filter(columns).table()

words (string),num (int),animals (string),thing (string),two strings (string),filter (string),num 2 (string),col_array (array),col_int (array),new_col_1 (int)
⸱⸱I⸱like⸱⸱⸱⸱⸱fish⸱⸱,1,dog,housé,cat-car,a,1,"['baby',⸱'sorry']","[1,⸱2,⸱3]",1
⸱⸱⸱⸱zombies,2,cat,tv,dog-tv,b,2,"['baby⸱1',⸱'sorry⸱1']","[3,⸱4]",1
simpsons⸱⸱⸱cat⸱lady,2,frog,table,eagle-tv-plus,1,3,"['baby⸱2',⸱'sorry⸱2']","[5,⸱6,⸱7]",1
None,3,eagle,glass,lion-pc,c,4,"['baby⸱3',⸱'sorry⸱3']","[7,⸱8]",1


table function took 1505.715 ms
filter function took 11.931 ms


words (string),num (int),animals (string),thing (string)
⸱⸱I⸱like⸱⸱⸱⸱⸱fish⸱⸱,1,dog,housé
⸱⸱⸱⸱zombies,2,cat,tv
simpsons⸱⸱⸱cat⸱lady,2,frog,table
None,3,eagle,glass


table function took 1475.297 ms


In [10]:
df.cols.filter("n.*", regex = True).table()

filter function took 10.935 ms


num (int),num 2 (string),new_col_1 (int)
1,1,1
2,2,1
2,3,1
3,4,1


table function took 1530.394 ms


In [11]:
df.cols.filter("*", data_type = "str").table()
df.table()

filter function took 14.959 ms


thing (string),words (string),animals (string),num 2 (string),filter (string),two strings (string)
housé,⸱⸱I⸱like⸱⸱⸱⸱⸱fish⸱⸱,dog,1,a,cat-car
tv,⸱⸱⸱⸱zombies,cat,2,b,dog-tv
table,simpsons⸱⸱⸱cat⸱lady,frog,3,1,eagle-tv-plus
glass,None,eagle,4,c,lion-pc


table function took 1512.032 ms


words (string),num (int),animals (string),thing (string),two strings (string),filter (string),num 2 (string),col_array (array),col_int (array),new_col_1 (int)
⸱⸱I⸱like⸱⸱⸱⸱⸱fish⸱⸱,1,dog,housé,cat-car,a,1,"['baby',⸱'sorry']","[1,⸱2,⸱3]",1
⸱⸱⸱⸱zombies,2,cat,tv,dog-tv,b,2,"['baby⸱1',⸱'sorry⸱1']","[3,⸱4]",1
simpsons⸱⸱⸱cat⸱lady,2,frog,table,eagle-tv-plus,1,3,"['baby⸱2',⸱'sorry⸱2']","[5,⸱6,⸱7]",1
None,3,eagle,glass,lion-pc,c,4,"['baby⸱3',⸱'sorry⸱3']","[7,⸱8]",1


table function took 1572.854 ms


## Rename Column
### Spark
You can not rename multiple columns using Spark Vanilla API


### Pandas
* Almost the same behavior https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.rename.html

In [12]:
df.cols.rename([('num','number')]).table()

rename function took 3.989 ms


words (string),number (int),animals (string),thing (string),two strings (string),filter (string),num 2 (string),col_array (array),col_int (array),new_col_1 (int)
⸱⸱I⸱like⸱⸱⸱⸱⸱fish⸱⸱,1,dog,housé,cat-car,a,1,"['baby',⸱'sorry']","[1,⸱2,⸱3]",1
⸱⸱⸱⸱zombies,2,cat,tv,dog-tv,b,2,"['baby⸱1',⸱'sorry⸱1']","[3,⸱4]",1
simpsons⸱⸱⸱cat⸱lady,2,frog,table,eagle-tv-plus,1,3,"['baby⸱2',⸱'sorry⸱2']","[5,⸱6,⸱7]",1
None,3,eagle,glass,lion-pc,c,4,"['baby⸱3',⸱'sorry⸱3']","[7,⸱8]",1


table function took 1519.962 ms


In [13]:
df.cols.rename(func = str.lower).table()

rename function took 26.892 ms


words (string),num (int),animals (string),thing (string),two strings (string),filter (string),num 2 (string),col_array (array),col_int (array),new_col_1 (int)
⸱⸱I⸱like⸱⸱⸱⸱⸱fish⸱⸱,1,dog,housé,cat-car,a,1,"['baby',⸱'sorry']","[1,⸱2,⸱3]",1
⸱⸱⸱⸱zombies,2,cat,tv,dog-tv,b,2,"['baby⸱1',⸱'sorry⸱1']","[3,⸱4]",1
simpsons⸱⸱⸱cat⸱lady,2,frog,table,eagle-tv-plus,1,3,"['baby⸱2',⸱'sorry⸱2']","[5,⸱6,⸱7]",1
None,3,eagle,glass,lion-pc,c,4,"['baby⸱3',⸱'sorry⸱3']","[7,⸱8]",1


table function took 1524.164 ms


In [14]:
df.cols.rename(func = str.upper).table()

rename function took 40.919 ms


WORDS (string),NUM (int),ANIMALS (string),THING (string),TWO STRINGS (string),FILTER (string),NUM 2 (string),COL_ARRAY (array),COL_INT (array),NEW_COL_1 (int)
⸱⸱I⸱like⸱⸱⸱⸱⸱fish⸱⸱,1,dog,housé,cat-car,a,1,"['baby',⸱'sorry']","[1,⸱2,⸱3]",1
⸱⸱⸱⸱zombies,2,cat,tv,dog-tv,b,2,"['baby⸱1',⸱'sorry⸱1']","[3,⸱4]",1
simpsons⸱⸱⸱cat⸱lady,2,frog,table,eagle-tv-plus,1,3,"['baby⸱2',⸱'sorry⸱2']","[5,⸱6,⸱7]",1
None,3,eagle,glass,lion-pc,c,4,"['baby⸱3',⸱'sorry⸱3']","[7,⸱8]",1


table function took 1500.441 ms


## Cast a columns

### Spark
* Can not cast multiple columns

### Pandas
This is a opinionated way to handle column casting. 
One of the first thing that every data cleaning process need to acomplish is define a data dictionary.
Because of that we prefer to create a tuple like this:

df.cols().cast(
[("words","str"),
("num","int"),
("animals","float"),
("thing","str")]
)


https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.astype.html

In [15]:
df.cols.cast([("num", "string"),("num 2", "integer")]).dtypes
#[('string',), ('integer',)]

cast function took 20.942 ms


[('words', 'string'),
 ('num', 'string'),
 ('animals', 'string'),
 ('thing', 'string'),
 ('two strings', 'string'),
 ('filter', 'string'),
 ('num 2', 'int'),
 ('col_array', 'array<string>'),
 ('col_int', 'array<int>'),
 ('new_col_1', 'int')]

In [16]:
df.cols.cast("num", "string").dtypes

cast function took 6.944 ms


[('words', 'string'),
 ('num', 'string'),
 ('animals', 'string'),
 ('thing', 'string'),
 ('two strings', 'string'),
 ('filter', 'string'),
 ('num 2', 'string'),
 ('col_array', 'array<string>'),
 ('col_int', 'array<int>'),
 ('new_col_1', 'int')]

In [17]:
df.cols.cast("*", "string").dtypes

cast function took 75.798 ms


[('words', 'string'),
 ('num', 'string'),
 ('animals', 'string'),
 ('thing', 'string'),
 ('two strings', 'string'),
 ('filter', 'string'),
 ('num 2', 'string'),
 ('col_array', 'string'),
 ('col_int', 'string'),
 ('new_col_1', 'string')]

In [18]:
list(zip(['words', 'num', 'animals', 'thing', 'two strings', 'filter', 'num 2', 'col_array', 'col_int'], ('string', 'string', 'string', 'string', 'string', 'string', 'string', 'string', 'string')))

[('words', 'string'),
 ('num', 'string'),
 ('animals', 'string'),
 ('thing', 'string'),
 ('two strings', 'string'),
 ('filter', 'string'),
 ('num 2', 'string'),
 ('col_array', 'string'),
 ('col_int', 'string')]

In [19]:
from pyspark.ml.linalg import Vectors

df.cols.cast("col_int", Vectors)

cast function took 27.886 ms


DataFrame[words: string, num: int, animals: string, thing: string, two strings: string, filter: string, num 2: string, col_array: array<string>, col_int: vector, new_col_1: int]

## Keep columns
### Spark
* You can not remove multiple columns

### Pandas
* Handle in pandas with drop


In [20]:
from pyspark.sql.functions import *
df.withColumn("num", col("num").cast(StringType()))


DataFrame[words: string, num: string, animals: string, thing: string, two strings: string, filter: string, num 2: string, col_array: array<string>, col_int: array<int>, new_col_1: int]

In [21]:
df.table()
df.cols.keep("num").table()

words (string),num (int),animals (string),thing (string),two strings (string),filter (string),num 2 (string),col_array (array),col_int (array),new_col_1 (int)
⸱⸱I⸱like⸱⸱⸱⸱⸱fish⸱⸱,1,dog,housé,cat-car,a,1,"['baby',⸱'sorry']","[1,⸱2,⸱3]",1
⸱⸱⸱⸱zombies,2,cat,tv,dog-tv,b,2,"['baby⸱1',⸱'sorry⸱1']","[3,⸱4]",1
simpsons⸱⸱⸱cat⸱lady,2,frog,table,eagle-tv-plus,1,3,"['baby⸱2',⸱'sorry⸱2']","[5,⸱6,⸱7]",1
None,3,eagle,glass,lion-pc,c,4,"['baby⸱3',⸱'sorry⸱3']","[7,⸱8]",1


table function took 1484.157 ms
keep function took 5.983 ms


num (int)
1
2
2
3


table function took 1511.533 ms


## Move columns
### Spark
Do not exist in spark

### Pandas
Do not exist in pandas

In [22]:
df.cols.move("words", "thing", "after").table()

move function took 24.964 ms


num (int),animals (string),thing (string),words (string),two strings (string),filter (string),num 2 (string),col_array (array),col_int (array),new_col_1 (int)
1,dog,housé,⸱⸱I⸱like⸱⸱⸱⸱⸱fish⸱⸱,cat-car,a,1,"['baby',⸱'sorry']","[1,⸱2,⸱3]",1
2,cat,tv,⸱⸱⸱⸱zombies,dog-tv,b,2,"['baby⸱1',⸱'sorry⸱1']","[3,⸱4]",1
2,frog,table,simpsons⸱⸱⸱cat⸱lady,eagle-tv-plus,1,3,"['baby⸱2',⸱'sorry⸱2']","[5,⸱6,⸱7]",1
3,eagle,glass,None,lion-pc,c,4,"['baby⸱3',⸱'sorry⸱3']","[7,⸱8]",1


table function took 1521.363 ms


## Sorting Columns
### Spark
You can not sort columns using Spark Vanilla API 

### Pandas
Similar to pandas
http://pandas.pydata.org/pandas-docs/version/0.19/generated/pandas.DataFrame.sort_values.html#pandas.DataFrame.sort_values

In [23]:
df.cols.sort().table()

sort function took 27.925 ms


animals (string),col_array (array),col_int (array),filter (string),new_col_1 (int),num (int),num 2 (string),thing (string),two strings (string),words (string)
dog,"['baby',⸱'sorry']","[1,⸱2,⸱3]",a,1,1,1,housé,cat-car,⸱⸱I⸱like⸱⸱⸱⸱⸱fish⸱⸱
cat,"['baby⸱1',⸱'sorry⸱1']","[3,⸱4]",b,1,2,2,tv,dog-tv,⸱⸱⸱⸱zombies
frog,"['baby⸱2',⸱'sorry⸱2']","[5,⸱6,⸱7]",1,1,2,3,table,eagle-tv-plus,simpsons⸱⸱⸱cat⸱lady
eagle,"['baby⸱3',⸱'sorry⸱3']","[7,⸱8]",c,1,3,4,glass,lion-pc,None


table function took 1508.342 ms


In [24]:
df.cols.sort(order = "desc").table()

sort function took 36.899 ms


words (string),two strings (string),thing (string),num 2 (string),num (int),new_col_1 (int),filter (string),col_int (array),col_array (array),animals (string)
⸱⸱I⸱like⸱⸱⸱⸱⸱fish⸱⸱,cat-car,housé,1,1,1,a,"[1,⸱2,⸱3]","['baby',⸱'sorry']",dog
⸱⸱⸱⸱zombies,dog-tv,tv,2,2,1,b,"[3,⸱4]","['baby⸱1',⸱'sorry⸱1']",cat
simpsons⸱⸱⸱cat⸱lady,eagle-tv-plus,table,3,2,1,1,"[5,⸱6,⸱7]","['baby⸱2',⸱'sorry⸱2']",frog
None,lion-pc,glass,4,3,1,c,"[7,⸱8]","['baby⸱3',⸱'sorry⸱3']",eagle


table function took 1562.474 ms


## Drop columns
### Spark 
* You can not delete multiple colums

### Pandas
* Almost the same as pandas
https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.drop.html

In [25]:
df2 = df.cols.drop("num")
df2 = df.cols.drop(["num","words"])
df2.table()

drop function took 3.934 ms
drop function took 3.965 ms


animals (string),thing (string),two strings (string),filter (string),num 2 (string),col_array (array),col_int (array),new_col_1 (int)
dog,housé,cat-car,a,1,"['baby',⸱'sorry']","[1,⸱2,⸱3]",1
cat,tv,dog-tv,b,2,"['baby⸱1',⸱'sorry⸱1']","[3,⸱4]",1
frog,table,eagle-tv-plus,1,3,"['baby⸱2',⸱'sorry⸱2']","[5,⸱6,⸱7]",1
eagle,glass,lion-pc,c,4,"['baby⸱3',⸱'sorry⸱3']","[7,⸱8]",1


table function took 1505.179 ms


In [26]:
df.table()

words (string),num (int),animals (string),thing (string),two strings (string),filter (string),num 2 (string),col_array (array),col_int (array),new_col_1 (int)
⸱⸱I⸱like⸱⸱⸱⸱⸱fish⸱⸱,1,dog,housé,cat-car,a,1,"['baby',⸱'sorry']","[1,⸱2,⸱3]",1
⸱⸱⸱⸱zombies,2,cat,tv,dog-tv,b,2,"['baby⸱1',⸱'sorry⸱1']","[3,⸱4]",1
simpsons⸱⸱⸱cat⸱lady,2,frog,table,eagle-tv-plus,1,3,"['baby⸱2',⸱'sorry⸱2']","[5,⸱6,⸱7]",1
None,3,eagle,glass,lion-pc,c,4,"['baby⸱3',⸱'sorry⸱3']","[7,⸱8]",1


table function took 1589.510 ms


## Chaining

cols y rows functions are used to organize and encapsulate optimus' functionality apart of Apache Spark Dataframe API. This have a disadvantage at chaining time because we need to user invoke cols or rows in every step.

At the same time it can be helpfull when you look at the code because every line is self explained.

In [27]:
df\
    .cols.rename([('num','number')])\
    .cols.drop(["number","words"])\
    .withColumn("new_col_2", lit("spongebob"))\
    .cols.append("new_col_1", 1)\
    .cols.sort(order= "desc")\
    .table()

rename function took 1.997 ms
drop function took 3.990 ms
append function took 7.949 ms
sort function took 23.902 ms


two strings (string),thing (string),num 2 (string),new_col_2 (string),new_col_1 (int),filter (string),col_int (array),col_array (array),animals (string)
cat-car,housé,1,spongebob,1,a,"[1,⸱2,⸱3]","['baby',⸱'sorry']",dog
dog-tv,tv,2,spongebob,1,b,"[3,⸱4]","['baby⸱1',⸱'sorry⸱1']",cat
eagle-tv-plus,table,3,spongebob,1,1,"[5,⸱6,⸱7]","['baby⸱2',⸱'sorry⸱2']",frog
lion-pc,glass,4,spongebob,1,c,"[7,⸱8]","['baby⸱3',⸱'sorry⸱3']",eagle


table function took 1505.452 ms


## Unnest  Columns
### Spark

### Pandas

In [74]:
df.table()
df.cols.unnest("two strings","-")\
    .table()

<class 'list'>


words (string),num (int),animals (string),thing (string),two strings (string),filter (string),num 2 (string),col_array (array),col_int (array),new_col_1 (int)
⸱⸱I⸱like⸱⸱⸱⸱⸱fish⸱⸱,1,dog,housé,cat-car,a,1,"['baby',⸱'sorry']","[1,⸱2,⸱3]",1
⸱⸱⸱⸱zombies,2,cat,tv,dog-tv,b,2,"['baby⸱1',⸱'sorry⸱1']","[3,⸱4]",1
simpsons⸱⸱⸱cat⸱lady,2,frog,table,eagle-tv-plus,1,3,"['baby⸱2',⸱'sorry⸱2']","[5,⸱6,⸱7]",1
None,3,eagle,glass,lion-pc,c,4,"['baby⸱3',⸱'sorry⸱3']","[7,⸱8]",1


table function took 1594.573 ms
filter function took 5.987 ms
cell function took 1502.186 ms
unnest function took 1560.030 ms
<class 'list'>


words (string),num (int),animals (string),thing (string),two strings (string),filter (string),num 2 (string),col_array (array),col_int (array),new_col_1 (int),two strings_0 (string),two strings_1 (string)
⸱⸱I⸱like⸱⸱⸱⸱⸱fish⸱⸱,1,dog,housé,cat-car,a,1,"['baby',⸱'sorry']","[1,⸱2,⸱3]",1,cat,car
⸱⸱⸱⸱zombies,2,cat,tv,dog-tv,b,2,"['baby⸱1',⸱'sorry⸱1']","[3,⸱4]",1,dog,tv
simpsons⸱⸱⸱cat⸱lady,2,frog,table,eagle-tv-plus,1,3,"['baby⸱2',⸱'sorry⸱2']","[5,⸱6,⸱7]",1,eagle,tv
None,3,eagle,glass,lion-pc,c,4,"['baby⸱3',⸱'sorry⸱3']","[7,⸱8]",1,lion,pc


table function took 1555.892 ms


In [72]:
df.cols.unnest("two strings","-", index = 1).table()

filter function took 5.984 ms
cell function took 1621.352 ms
unnest function took 1741.019 ms
<class 'list'>


words (string),num (int),animals (string),thing (string),two strings (string),filter (string),num 2 (string),col_array (array),col_int (array),new_col_1 (int),two strings_1 (string)
⸱⸱I⸱like⸱⸱⸱⸱⸱fish⸱⸱,1,dog,housé,cat-car,a,1,"['baby',⸱'sorry']","[1,⸱2,⸱3]",1,car
⸱⸱⸱⸱zombies,2,cat,tv,dog-tv,b,2,"['baby⸱1',⸱'sorry⸱1']","[3,⸱4]",1,tv
simpsons⸱⸱⸱cat⸱lady,2,frog,table,eagle-tv-plus,1,3,"['baby⸱2',⸱'sorry⸱2']","[5,⸱6,⸱7]",1,tv
None,3,eagle,glass,lion-pc,c,4,"['baby⸱3',⸱'sorry⸱3']","[7,⸱8]",1,pc


table function took 1547.277 ms


## Impute
### Spark

In [30]:
df_cast =df.cols.cast([("num","double"),("num 2", "double")])
df_cast.dtypes

cast function took 15.958 ms


[('words', 'string'),
 ('num', 'double'),
 ('animals', 'string'),
 ('thing', 'string'),
 ('two strings', 'string'),
 ('filter', 'string'),
 ('num 2', 'double'),
 ('col_array', 'array<string>'),
 ('col_int', 'array<int>'),
 ('new_col_1', 'int')]

In [31]:
df_cast.cols.impute(["num","num 2"], ["out_a","out_B"], "mean")
df_cast.table()

impute function took 1522.893 ms


words (string),num (double),animals (string),thing (string),two strings (string),filter (string),num 2 (double),col_array (array),col_int (array),new_col_1 (int)
⸱⸱I⸱like⸱⸱⸱⸱⸱fish⸱⸱,1.0,dog,housé,cat-car,a,1.0,"['baby',⸱'sorry']","[1,⸱2,⸱3]",1
⸱⸱⸱⸱zombies,2.0,cat,tv,dog-tv,b,2.0,"['baby⸱1',⸱'sorry⸱1']","[3,⸱4]",1
simpsons⸱⸱⸱cat⸱lady,2.0,frog,table,eagle-tv-plus,1,3.0,"['baby⸱2',⸱'sorry⸱2']","[5,⸱6,⸱7]",1
None,3.0,eagle,glass,lion-pc,c,4.0,"['baby⸱3',⸱'sorry⸱3']","[7,⸱8]",1


table function took 1650.138 ms


## Get columns by type
### Spark

### Pandas

In [32]:
df.cols.filter_by_dtypes("int").table()

filter_by_dtypes function took 6.946 ms


num (int),new_col_1 (int)
1,1
2,1
2,1
3,1


table function took 1516.284 ms


## Apply custom function
### Spark
You need to declare a UDF Spark function

### Pandas
Almost the same behavior

In [33]:
df.table()

words (string),num (int),animals (string),thing (string),two strings (string),filter (string),num 2 (string),col_array (array),col_int (array),new_col_1 (int)
⸱⸱I⸱like⸱⸱⸱⸱⸱fish⸱⸱,1,dog,housé,cat-car,a,1,"['baby',⸱'sorry']","[1,⸱2,⸱3]",1
⸱⸱⸱⸱zombies,2,cat,tv,dog-tv,b,2,"['baby⸱1',⸱'sorry⸱1']","[3,⸱4]",1
simpsons⸱⸱⸱cat⸱lady,2,frog,table,eagle-tv-plus,1,3,"['baby⸱2',⸱'sorry⸱2']","[5,⸱6,⸱7]",1
None,3,eagle,glass,lion-pc,c,4,"['baby⸱3',⸱'sorry⸱3']","[7,⸱8]",1


table function took 1544.931 ms


In [34]:
def sum_(val, attr):
    return attr

df.cols.apply_by_dtypes("filter", sum_, "string", "10", data_type="integer").table()

Using 'Pandas UDF' to process column 'filter'
apply function took 30.947 ms
apply_by_dtypes function took 68.802 ms


words (string),num (int),animals (string),thing (string),two strings (string),filter (string),num 2 (string),col_array (array),col_int (array),new_col_1 (int)
⸱⸱I⸱like⸱⸱⸱⸱⸱fish⸱⸱,1,dog,housé,cat-car,a,1,"['baby',⸱'sorry']","[1,⸱2,⸱3]",1
⸱⸱⸱⸱zombies,2,cat,tv,dog-tv,b,2,"['baby⸱1',⸱'sorry⸱1']","[3,⸱4]",1
simpsons⸱⸱⸱cat⸱lady,2,frog,table,eagle-tv-plus,10,3,"['baby⸱2',⸱'sorry⸱2']","[5,⸱6,⸱7]",1
None,3,eagle,glass,lion-pc,c,4,"['baby⸱3',⸱'sorry⸱3']","[7,⸱8]",1


table function took 2925.418 ms


In [35]:
def func(val, attr):
    return val + attr

df.cols.apply(["num", "num"], func, "int", 32 ,"udf").table()

Using 'UDF' to process column 'num'
Using 'UDF' to process column 'num'
apply function took 78.820 ms


words (string),num (int),animals (string),thing (string),two strings (string),filter (string),num 2 (string),col_array (array),col_int (array),new_col_1 (int)
⸱⸱I⸱like⸱⸱⸱⸱⸱fish⸱⸱,65,dog,housé,cat-car,a,1,"['baby',⸱'sorry']","[1,⸱2,⸱3]",1
⸱⸱⸱⸱zombies,66,cat,tv,dog-tv,b,2,"['baby⸱1',⸱'sorry⸱1']","[3,⸱4]",1
simpsons⸱⸱⸱cat⸱lady,66,frog,table,eagle-tv-plus,1,3,"['baby⸱2',⸱'sorry⸱2']","[5,⸱6,⸱7]",1
None,67,eagle,glass,lion-pc,c,4,"['baby⸱3',⸱'sorry⸱3']","[7,⸱8]",1


table function took 1968.546 ms


In [67]:
from optimus.functions import filter_row_by_data_type as fbdt

df.where(fbdt("filter", "integer")).table()

<class 'list'>


words (string),num (int),animals (string),thing (string),two strings (string),filter (string),num 2 (string),col_array (array),col_int (array),new_col_1 (int)
simpsons⸱⸱⸱cat⸱lady,2,frog,table,eagle-tv-plus,1,3,"['baby⸱2',⸱'sorry⸱2']","[5,⸱6,⸱7]",1


table function took 6133.127 ms


In [39]:
from optimus.functions import abstract_udf as audf 

def func(val, attr):
    return val>1

df.where(audf("num", func, "boolean")).table()

Using 'Pandas UDF' to process column 'num'


words (string),num (int),animals (string),thing (string),two strings (string),filter (string),num 2 (string),col_array (array),col_int (array),new_col_1 (int)
⸱⸱⸱⸱zombies,2,cat,tv,dog-tv,b,2,"['baby⸱1',⸱'sorry⸱1']","[3,⸱4]",1
simpsons⸱⸱⸱cat⸱lady,2,frog,table,eagle-tv-plus,1,3,"['baby⸱2',⸱'sorry⸱2']","[5,⸱6,⸱7]",1
None,3,eagle,glass,lion-pc,c,4,"['baby⸱3',⸱'sorry⸱3']","[7,⸱8]",1


table function took 2501.348 ms


In [40]:
def func(val, attr):
    return val + attr

df.cols.apply(["num", "num"], func, "int", 10).table()

Using 'Pandas UDF' to process column 'num'
Using 'Pandas UDF' to process column 'num'
apply function took 71.773 ms


words (string),num (int),animals (string),thing (string),two strings (string),filter (string),num 2 (string),col_array (array),col_int (array),new_col_1 (int)
⸱⸱I⸱like⸱⸱⸱⸱⸱fish⸱⸱,21,dog,housé,cat-car,a,1,"['baby',⸱'sorry']","[1,⸱2,⸱3]",1
⸱⸱⸱⸱zombies,22,cat,tv,dog-tv,b,2,"['baby⸱1',⸱'sorry⸱1']","[3,⸱4]",1
simpsons⸱⸱⸱cat⸱lady,22,frog,table,eagle-tv-plus,1,3,"['baby⸱2',⸱'sorry⸱2']","[5,⸱6,⸱7]",1
None,23,eagle,glass,lion-pc,c,4,"['baby⸱3',⸱'sorry⸱3']","[7,⸱8]",1


table function took 2546.514 ms


In [41]:
# https://stackoverflow.com/questions/31400143/column-filtering-in-pyspark
from optimus.functions import abstract_udf as audf 

def func(val, attr):
    return val+attr[0]

df.withColumn("num", audf ("num", func, "int", [10,20])).table()

Using 'Pandas UDF' to process column 'num'


words (string),num (int),animals (string),thing (string),two strings (string),filter (string),num 2 (string),col_array (array),col_int (array),new_col_1 (int)
⸱⸱I⸱like⸱⸱⸱⸱⸱fish⸱⸱,11,dog,housé,cat-car,a,1,"['baby',⸱'sorry']","[1,⸱2,⸱3]",1
⸱⸱⸱⸱zombies,12,cat,tv,dog-tv,b,2,"['baby⸱1',⸱'sorry⸱1']","[3,⸱4]",1
simpsons⸱⸱⸱cat⸱lady,12,frog,table,eagle-tv-plus,1,3,"['baby⸱2',⸱'sorry⸱2']","[5,⸱6,⸱7]",1
None,13,eagle,glass,lion-pc,c,4,"['baby⸱3',⸱'sorry⸱3']","[7,⸱8]",1


table function took 2478.721 ms


In [42]:
from pyspark.sql import functions as F
def func(col_name, attr):
    # return F.col(col_name) + 1
    return F.when(F.col(col_name)>0 ,2)

df.cols.apply_exp(["num","num 2"], func).table()

Using 'Column Expression' to process column 'num'
Using 'Column Expression' to process column 'num 2'
apply_exp function took 103.686 ms


words (string),num (int),animals (string),thing (string),two strings (string),filter (string),num 2 (int),col_array (array),col_int (array),new_col_1 (int)
⸱⸱I⸱like⸱⸱⸱⸱⸱fish⸱⸱,2,dog,housé,cat-car,a,2,"['baby',⸱'sorry']","[1,⸱2,⸱3]",1
⸱⸱⸱⸱zombies,2,cat,tv,dog-tv,b,2,"['baby⸱1',⸱'sorry⸱1']","[3,⸱4]",1
simpsons⸱⸱⸱cat⸱lady,2,frog,table,eagle-tv-plus,1,2,"['baby⸱2',⸱'sorry⸱2']","[5,⸱6,⸱7]",1
None,2,eagle,glass,lion-pc,c,2,"['baby⸱3',⸱'sorry⸱3']","[7,⸱8]",1


table function took 1543.996 ms


In [43]:
from pyspark.sql import functions as F
def func(col_name, attr):
    return F.upper(F.col(col_name))

df.cols.apply_exp(["two strings","animals"], func).table()

Using 'Column Expression' to process column 'two strings'
Using 'Column Expression' to process column 'animals'
apply_exp function took 17.919 ms


words (string),num (int),animals (string),thing (string),two strings (string),filter (string),num 2 (string),col_array (array),col_int (array),new_col_1 (int)
⸱⸱I⸱like⸱⸱⸱⸱⸱fish⸱⸱,1,DOG,housé,CAT-CAR,a,1,"['baby',⸱'sorry']","[1,⸱2,⸱3]",1
⸱⸱⸱⸱zombies,2,CAT,tv,DOG-TV,b,2,"['baby⸱1',⸱'sorry⸱1']","[3,⸱4]",1
simpsons⸱⸱⸱cat⸱lady,2,FROG,table,EAGLE-TV-PLUS,1,3,"['baby⸱2',⸱'sorry⸱2']","[5,⸱6,⸱7]",1
None,3,EAGLE,glass,LION-PC,c,4,"['baby⸱3',⸱'sorry⸱3']","[7,⸱8]",1


table function took 1501.224 ms


In [44]:
def func(val, attr):
    return 10

col = "num"

df.cols.apply(col, func, "int", when= df["num"]>1).table()

df.cols.apply(col, func, "int", when= fbdt(col, "int")).table()

Using 'Pandas UDF' to process column 'num'
apply function took 36.701 ms


words (string),num (int),animals (string),thing (string),two strings (string),filter (string),num 2 (string),col_array (array),col_int (array),new_col_1 (int)
⸱⸱I⸱like⸱⸱⸱⸱⸱fish⸱⸱,1,dog,housé,cat-car,a,1,"['baby',⸱'sorry']","[1,⸱2,⸱3]",1
⸱⸱⸱⸱zombies,10,cat,tv,dog-tv,b,2,"['baby⸱1',⸱'sorry⸱1']","[3,⸱4]",1
simpsons⸱⸱⸱cat⸱lady,10,frog,table,eagle-tv-plus,1,3,"['baby⸱2',⸱'sorry⸱2']","[5,⸱6,⸱7]",1
None,10,eagle,glass,lion-pc,c,4,"['baby⸱3',⸱'sorry⸱3']","[7,⸱8]",1


table function took 2484.872 ms
Using 'Pandas UDF' to process column 'num'
apply function took 39.924 ms


words (string),num (int),animals (string),thing (string),two strings (string),filter (string),num 2 (string),col_array (array),col_int (array),new_col_1 (int)
⸱⸱I⸱like⸱⸱⸱⸱⸱fish⸱⸱,10,dog,housé,cat-car,a,1,"['baby',⸱'sorry']","[1,⸱2,⸱3]",1
⸱⸱⸱⸱zombies,10,cat,tv,dog-tv,b,2,"['baby⸱1',⸱'sorry⸱1']","[3,⸱4]",1
simpsons⸱⸱⸱cat⸱lady,10,frog,table,eagle-tv-plus,1,3,"['baby⸱2',⸱'sorry⸱2']","[5,⸱6,⸱7]",1
None,10,eagle,glass,lion-pc,c,4,"['baby⸱3',⸱'sorry⸱3']","[7,⸱8]",1


table function took 2627.859 ms


## Count Nulls
### Spark

### Pandas

In [45]:
import numpy as np

df_null = op.spark.createDataFrame(
    [(1, 1, None), (1, 2, float(5)), (1, 3, np.nan), (1, 4, None), (1, 5, float(10)), (1, 6, float('nan')), (1, 6, float('nan'))],
    ('session', "timestamp1", "id2"))

In [46]:
df_null.cols.count_na("id2")

count_na function took 1562.988 ms


{'id2': 5}

In [47]:
df_null.cols.count_na("*")

count_na function took 1632.170 ms


{'id2': 5, 'session': 0, 'timestamp1': 0}

## Count uniques
### Spark

### Pandas


In [48]:
df.cols.count_uniques("*")

count_uniques function took 2103.842 ms


{'approx_count_distinct_animals': 4,
 'approx_count_distinct_col_array': 3,
 'approx_count_distinct_col_int': 4,
 'approx_count_distinct_filter': 4,
 'approx_count_distinct_new_col_1': 1,
 'approx_count_distinct_num': 3,
 'approx_count_distinct_num 2': 4,
 'approx_count_distinct_thing': 4,
 'approx_count_distinct_two strings': 4,
 'approx_count_distinct_words': 3}

## Unique
### Spark
An abstraction of distinct to be use in multiple columns at the same time

### Pandas
Similar behavior than pandas

In [49]:
df.table()

words (string),num (int),animals (string),thing (string),two strings (string),filter (string),num 2 (string),col_array (array),col_int (array),new_col_1 (int)
⸱⸱I⸱like⸱⸱⸱⸱⸱fish⸱⸱,1,dog,housé,cat-car,a,1,"['baby',⸱'sorry']","[1,⸱2,⸱3]",1
⸱⸱⸱⸱zombies,2,cat,tv,dog-tv,b,2,"['baby⸱1',⸱'sorry⸱1']","[3,⸱4]",1
simpsons⸱⸱⸱cat⸱lady,2,frog,table,eagle-tv-plus,1,3,"['baby⸱2',⸱'sorry⸱2']","[5,⸱6,⸱7]",1
None,3,eagle,glass,lion-pc,c,4,"['baby⸱3',⸱'sorry⸱3']","[7,⸱8]",1


table function took 1477.845 ms


In [50]:
df_distinct = op.create.df(
            [
                ("words", "str", True),
                ("num", "int", True)
            ],
[
                ("  I like     fish  ", 1),
                ("    zombies", 2),
                ("simpsons   cat lady", 2),
                (None, 3),
                  (None, 0)
            ])

In [51]:
df_distinct.cols.unique("num").table()

unique function took 9.938 ms


num (int)
1
3
2
0


table function took 2354.539 ms


## Count Zeros

In [52]:
df_zeros = df_distinct
df_zeros.table()
df_zeros.cols.count_zeros("*")

words (string),num (int)
⸱⸱I⸱like⸱⸱⸱⸱⸱fish⸱⸱,1
⸱⸱⸱⸱zombies,2
simpsons⸱⸱⸱cat⸱lady,2
None,3
None,0


table function took 1465.273 ms
count_zeros function took 1570.480 ms


{'num': 1, 'words': 0}

## Column Data Types

In [53]:
df.cols.dtypes('*')

dtypes function took 0.000 ms


{'animals': 'string',
 'col_array': 'array<string>',
 'col_int': 'array<int>',
 'filter': 'string',
 'new_col_1': 'int',
 'num': 'int',
 'num 2': 'string',
 'thing': 'string',
 'two strings': 'string',
 'words': 'string'}

## Replace

In [54]:
df.cols.replace(["two strings","animals"], ["dog-tv", "cat", "eagle", "fish"], "animals").table()
df.cols.replace('animals',["dog","cat"],"animals").table()

df.cols.replace('animals',[("dog","animals"),("cat","animals")]).table()

df.cols.replace('animals',"dog",'animal').table()

df.cols.replace('num',["3",2], 10).table()

df.cols.replace('num',[("3",6),(2,6)]).table()

dtypes function took 0.000 ms
dtypes function took 0.000 ms
replace function took 30.945 ms


words (string),num (int),animals (string),thing (string),two strings (string),filter (string),num 2 (string),col_array (array),col_int (array),new_col_1 (int)
⸱⸱I⸱like⸱⸱⸱⸱⸱fish⸱⸱,1,dog,housé,cat-car,a,1,"['baby',⸱'sorry']","[1,⸱2,⸱3]",1
⸱⸱⸱⸱zombies,2,animals,tv,animals,b,2,"['baby⸱1',⸱'sorry⸱1']","[3,⸱4]",1
simpsons⸱⸱⸱cat⸱lady,2,frog,table,eagle-tv-plus,1,3,"['baby⸱2',⸱'sorry⸱2']","[5,⸱6,⸱7]",1
None,3,animals,glass,lion-pc,c,4,"['baby⸱3',⸱'sorry⸱3']","[7,⸱8]",1


table function took 1554.300 ms
dtypes function took 0.000 ms
replace function took 10.970 ms


words (string),num (int),animals (string),thing (string),two strings (string),filter (string),num 2 (string),col_array (array),col_int (array),new_col_1 (int)
⸱⸱I⸱like⸱⸱⸱⸱⸱fish⸱⸱,1,animals,housé,cat-car,a,1,"['baby',⸱'sorry']","[1,⸱2,⸱3]",1
⸱⸱⸱⸱zombies,2,animals,tv,dog-tv,b,2,"['baby⸱1',⸱'sorry⸱1']","[3,⸱4]",1
simpsons⸱⸱⸱cat⸱lady,2,frog,table,eagle-tv-plus,1,3,"['baby⸱2',⸱'sorry⸱2']","[5,⸱6,⸱7]",1
None,3,eagle,glass,lion-pc,c,4,"['baby⸱3',⸱'sorry⸱3']","[7,⸱8]",1


table function took 1516.912 ms
dtypes function took 0.000 ms
replace function took 10.953 ms


...

table function took 1525.269 ms
replace function took 0.000 ms


words (string),num (int),animals (string),thing (string),two strings (string),filter (string),num 2 (string),col_array (array),col_int (array),new_col_1 (int)
⸱⸱I⸱like⸱⸱⸱⸱⸱fish⸱⸱,1,dog,housé,cat-car,a,1,"['baby',⸱'sorry']","[1,⸱2,⸱3]",1
⸱⸱⸱⸱zombies,2,cat,tv,dog-tv,b,2,"['baby⸱1',⸱'sorry⸱1']","[3,⸱4]",1
simpsons⸱⸱⸱cat⸱lady,2,frog,table,eagle-tv-plus,1,3,"['baby⸱2',⸱'sorry⸱2']","[5,⸱6,⸱7]",1
None,3,eagle,glass,lion-pc,c,4,"['baby⸱3',⸱'sorry⸱3']","[7,⸱8]",1


table function took 1498.800 ms
replace function took 0.000 ms


words (string),num (int),animals (string),thing (string),two strings (string),filter (string),num 2 (string),col_array (array),col_int (array),new_col_1 (int)
⸱⸱I⸱like⸱⸱⸱⸱⸱fish⸱⸱,1,dog,housé,cat-car,a,1,"['baby',⸱'sorry']","[1,⸱2,⸱3]",1
⸱⸱⸱⸱zombies,2,cat,tv,dog-tv,b,2,"['baby⸱1',⸱'sorry⸱1']","[3,⸱4]",1
simpsons⸱⸱⸱cat⸱lady,2,frog,table,eagle-tv-plus,1,3,"['baby⸱2',⸱'sorry⸱2']","[5,⸱6,⸱7]",1
None,3,eagle,glass,lion-pc,c,4,"['baby⸱3',⸱'sorry⸱3']","[7,⸱8]",1


table function took 1481.095 ms


In [55]:
df.cols.replace('*','.*[Cc]at.*', 'animal', regex=True).table()
#df.cols().replace('*','cat', 'animal', regex=True).show()


replace function took 136.657 ms


words (string),num (int),animals (string),thing (string),two strings (string),filter (string),num 2 (string),col_array (array),col_int (array),new_col_1 (int)
⸱⸱I⸱like⸱⸱⸱⸱⸱fish⸱⸱,1,dog,housé,animal,a,1,"['baby',⸱'sorry']","[1,⸱2,⸱3]",1
⸱⸱⸱⸱zombies,2,animal,tv,dog-tv,b,2,"['baby⸱1',⸱'sorry⸱1']","[3,⸱4]",1
animal,2,frog,table,eagle-tv-plus,1,3,"['baby⸱2',⸱'sorry⸱2']","[5,⸱6,⸱7]",1
None,3,eagle,glass,lion-pc,c,4,"['baby⸱3',⸱'sorry⸱3']","[7,⸱8]",1


table function took 1515.737 ms


## Nest

In [56]:
df.cols.nest(["num", "num"], "nested","sdf" ,"vector").table()

vector
nest function took 118.677 ms


words (string),num (int),animals (string),thing (string),two strings (string),filter (string),num 2 (string),col_array (array),col_int (array),new_col_1 (int),nested (vector)
⸱⸱I⸱like⸱⸱⸱⸱⸱fish⸱⸱,1,dog,housé,cat-car,a,1,"['baby',⸱'sorry']","[1,⸱2,⸱3]",1,"[1.0,1.0]"
⸱⸱⸱⸱zombies,2,cat,tv,dog-tv,b,2,"['baby⸱1',⸱'sorry⸱1']","[3,⸱4]",1,"[2.0,2.0]"
simpsons⸱⸱⸱cat⸱lady,2,frog,table,eagle-tv-plus,1,3,"['baby⸱2',⸱'sorry⸱2']","[5,⸱6,⸱7]",1,"[2.0,2.0]"
None,3,eagle,glass,lion-pc,c,4,"['baby⸱3',⸱'sorry⸱3']","[7,⸱8]",1,"[3.0,3.0]"


table function took 1670.000 ms


In [57]:
df.cols.nest(["animals", "two strings"], ["nested"], shape = "string").table()

string
Using 'Column Expression' to process column 'nested'
nest function took 14.956 ms


words (string),num (int),animals (string),thing (string),two strings (string),filter (string),num 2 (string),col_array (array),col_int (array),new_col_1 (int),nested (string)
⸱⸱I⸱like⸱⸱⸱⸱⸱fish⸱⸱,1,dog,housé,cat-car,a,1,"['baby',⸱'sorry']","[1,⸱2,⸱3]",1,None
⸱⸱⸱⸱zombies,2,cat,tv,dog-tv,b,2,"['baby⸱1',⸱'sorry⸱1']","[3,⸱4]",1,None
simpsons⸱⸱⸱cat⸱lady,2,frog,table,eagle-tv-plus,1,3,"['baby⸱2',⸱'sorry⸱2']","[5,⸱6,⸱7]",1,None
None,3,eagle,glass,lion-pc,c,4,"['baby⸱3',⸱'sorry⸱3']","[7,⸱8]",1,None


table function took 1578.440 ms


In [58]:
df.cols.nest(["animals", "two strings"], ["nested"],shape="array").table()

array
Using 'Column Expression' to process column 'nested'
nest function took 11.967 ms


words (string),num (int),animals (string),thing (string),two strings (string),filter (string),num 2 (string),col_array (array),col_int (array),new_col_1 (int),nested (array)
⸱⸱I⸱like⸱⸱⸱⸱⸱fish⸱⸱,1,dog,housé,cat-car,a,1,"['baby',⸱'sorry']","[1,⸱2,⸱3]",1,"['dog',⸱'cat-car']"
⸱⸱⸱⸱zombies,2,cat,tv,dog-tv,b,2,"['baby⸱1',⸱'sorry⸱1']","[3,⸱4]",1,"['cat',⸱'dog-tv']"
simpsons⸱⸱⸱cat⸱lady,2,frog,table,eagle-tv-plus,1,3,"['baby⸱2',⸱'sorry⸱2']","[5,⸱6,⸱7]",1,"['frog',⸱'eagle-tv-plus']"
None,3,eagle,glass,lion-pc,c,4,"['baby⸱3',⸱'sorry⸱3']","[7,⸱8]",1,"['eagle',⸱'lion-pc']"


table function took 1509.092 ms


## Unnest
Works with arrays, Vectors and strings(ala split)

In [59]:
df\
    .cols.unnest(["col_array"])\
    .cols.unnest(["col_int"])\
    .cols.unnest(["two strings"], n= 3, mark = "-")\
    .cols.filter("two.*", regex = True)\
    .table()

filter function took 5.978 ms
cell function took 1470.283 ms
unnest function took 1522.111 ms
filter function took 3.990 ms
cell function took 1454.637 ms
unnest function took 1511.518 ms
unnest function took 25.926 ms
filter function took 17.952 ms


thing (string),two strings (string),two strings_0 (string),two strings_1 (string),two strings_2 (string)
housé,cat-car,cat,car,None
tv,dog-tv,dog,tv,None
table,eagle-tv-plus,eagle,tv,plus
glass,lion-pc,lion,pc,None


table function took 1482.927 ms


In [60]:
from pyspark.ml.linalg import Vectors

df1 = op.sc.parallelize([
    ("assert", Vectors.dense([1, 2, 3])),
    ("require", Vectors.sparse(3, {1: 2}))
]).toDF(["word", "vector"])    

In [61]:
df1\
    .cols.unnest(["vector"])\
    .table()

unnest function took 1979.397 ms


word (string),vector (vector),_3 (double),_4 (double),_5 (double)
assert,"[1.0,2.0,3.0]",1.0,2.0,3.0
require,"(3,[1],[2.0])",0.0,2.0,0.0


table function took 1935.473 ms


## Hist

In [ ]:
df.cols.hist(["num 2"])

In [ ]:
df.table()